In [ ]:
import pandas as pd
import os
import numpy as np

In [ ]:
task = 'taskA'

In [ ]:
os.chdir('../dataset')

In [ ]:
os.makedirs('processed/taskA', exist_ok=True)
os.makedirs('processed/taskB', exist_ok=True)

In [ ]:
def process_df_cols(df):
    final_cols = {}
    drop_col = []
    for col in df.columns:
        if col.lower().__contains__('text'):
            final_cols[col] = 'text'
        elif col.lower().__contains__('category') or col.lower().__contains__('label'):
            final_cols[col] = 'category'
        else:
            drop_col.append(col)
    df.drop(columns=drop_col, inplace=True)
    df.rename(columns=final_cols, inplace=True)
    return df

def process_df(task):
    mapper = {
        'Non-anti-LGBT+ content': 'non-anti-LGBT',
        'Homophobia': 'homophobic',
        'Transphobia': 'transphobic',
        'Homophobic': 'homophobic',
        'Transphobic': 'transphobic',
        'None': 'non-anti-LGBT'
    } if task == 'taskA' else {
        'Counter-speech': 'counter-speech',
        'Homophobia': 'homophobic',
        'Homophobic-Threatening': 'homophobic-threatening',
        'Homophobic-derogation': 'homophobic-derogation',
        'Hope-Speech': 'hope-speech',
        'None': 'none-of-the-categories',
        'None of categories': 'none-of-the-categories',
        'None-of-the-above': 'none-of-the-categories',
        'Transphobia': 'transphobic',
        'Transphobic-Threatening': 'transphobic-threatening',
        'Transphobic-derogation': 'transphobic-derogation'
    }
    for file in os.listdir(task):
        try:
            if (task == 'taskA' and file.split('_')[1] in ['eng', 'hin', 'hindi', 'spanish', 'tam', 'mal']) or (task == 'taskB' and file.split('_')[1] in ['eng', 'tam', 'mal']):
                df = pd.read_csv(os.path.join(task, file))
                df = process_df_cols(df)
                df.fillna('None', inplace=True)
                filtered_rows = []
                for row in df.itertuples():
                    if len(row.text) < 5:
                        continue
                    filtered_rows.append({'text': row.text, 'category': mapper[row.category]})
                new_df = pd.DataFrame(filtered_rows)
                label_mapper = {k:v for v,k in enumerate(sorted(new_df.category.unique()))}
                print(label_mapper)
                new_df['label'] = new_df['category'].apply(lambda x: label_mapper[x])
                new_df.to_csv(os.path.join('processed', task, file), index=False)      
        except:
            print(file)  
            

In [ ]:
process_df('taskA')

In [ ]:
process_df('taskB')